In [4]:
from config import NUM_TRAINING_ITERATIONS, CONVERGENCE_THRESHOLD
from formulas import err
from models import Layer, cfile

f = None
curr_point = 0
target = []
attrs = []
total_runs = 0
data = None
num_incorrect = 0
prev_sample_err = 0
curr_sample_err = 0

def parse_data(fname):
    # reset all the data
    global curr_point
    global total_runs
    global target
    global attrs
    global num_incorrect
    global prev_sample_err
    global curr_sample_err
    global data
    global f

    curr_point = 0
    total_runs = 0
    target = []
    attrs = []
    num_incorrect = 0
    prev_sample_err = 0
    curr_sample_err = 0

    # set the proper data file
    data_file = 'err.txt'
    if fname == 'training.txt':
        data_file = 'training_err.txt'
    elif fname == 'validation.txt':
        data_file = 'validation_err.txt'
    elif fname == 'testing.txt':
        data_file = 'testing_err.txt'

    # clear the file
    open(data_file, 'w+').close()

    # open the data file for logging
    data = cfile(data_file, 'w')

    f = open(fname, 'r').readlines()

    for row in f:
        row = [x.strip() for x in row.split(',')]
        row = [int(num) for num in row]    
        target.append(int(row[1])) #row[0] represent p class, row[1] represent e class.
                                   #if you represent e and p classes into one elment (0 for p and 1 for e, etc.), please modifiy.
        attrs.append(row[2:])

if __name__ == '__main__':
    print ("Parsing the training dataset...")
    # parse the training dataset and store its information into globals
    parse_data('training.txt')

    # set up the layers to be used    
    x = Layer(6, attrs[curr_point], 1)
    y = Layer(3, x.layer_out, 2)

    print ("Begining training the neural network:")
    # iterate through to train the neural network
    while total_runs < NUM_TRAINING_ITERATIONS:   
        

        # set up the first layer and evaluate it
        x.input_vals = attrs[curr_point]
        x.eval()

        # set up the second layer and evaluate it
        y.input_vals = x.layer_out
        y.eval()

        # backpropogate
        y.backprop(target[curr_point])        
        x.backprop(y)

        # get the current error, that is equall to sum((target_output-current_output)^2/2)
        curr_err = err(y.layer_out[0], target[curr_point])

        # round up and down to check err
        if y.layer_out[0] >= 0.5:
            temp = 1
        else:
            temp = 0

        # increment the number incorrect if its wrong
        if(temp != target[curr_point]):
            num_incorrect += 1

        # check to see if we have converged
        if total_runs % 100 == 0:
            prev_sample_err = curr_sample_err
            curr_sample_err = curr_err
            if abs(prev_sample_err - curr_sample_err) < CONVERGENCE_THRESHOLD:
                print ("Data has converged at the " + str(total_runs) + "th run.")
                break;

        # print information about the current iteration
        print ("Current iteration: " + str(total_runs))
        print ("Current error: " + str(curr_err) + "\n")
        data.w(curr_err)

        # iterate
        total_runs += 1
        curr_point += 1

        if curr_point >= len(f):
            curr_point = 0

    # close the file
    data.close()

    print ("Neural network is done training! Hit enter to validation processing.")
    print ("Error percentage on training set: " + str(float(num_incorrect)/NUM_TRAINING_ITERATIONS))
    #raw_input()
    input()

    print ("Parsing the validation dataset...")
    # parse the validation dataset and store its information into globals
    parse_data('validation.txt')

    print ("Begining validating the neural network:")
    # iterate through to train the neural network
    while total_runs < len(f):
                             
        # set up the first layer and evaluate it
        x.input_vals = attrs[curr_point]
        x.eval()

        # set up the second layer and evaluate it
        y.input_vals = x.layer_out
        y.eval()

        # backpropogate
        y.backprop(target[curr_point])
        x.backprop(y)

        # get the current error
        #squared error, that is equall to sum((target_output - current output)^2 / 2))
        curr_err = err(y.layer_out[0], target[curr_point])

        # round up and down to check err
        if y.layer_out[0] >= 0.5:
            temp = 1
        else:
            temp = 0

        # increment the number incorrect if its wrong
        if(temp != target[curr_point]):
            num_incorrect += 1

        # check to see if we have converged
        if total_runs % 100 == 0:
            prev_sample_err = curr_sample_err
            curr_sample_err = curr_err
            if abs(prev_sample_err - curr_sample_err) < CONVERGENCE_THRESHOLD:
                print ("Data has converged at the " + str(total_runs) + "th run.")
                break;

        # print information about the current iteration
        print ("Current iteration: " + str(total_runs))
        print ("Current error: " + str(curr_err) + "\n")
        data.w(curr_err)

        # iterate
        total_runs += 1
        curr_point += 1

        if curr_point >= len(f):
            curr_point = 0

    # close the file
    data.close()

    print ("Neural network is done validating! Hit enter to test it.")
    print ("Error percentage on validation set: " + str(float(num_incorrect)/NUM_TRAINING_ITERATIONS))
    #raw_input()
    input()

    print ("Begin testing the neural network:")
    # parse the testing data and store its information into globals
    parse_data('testing.txt')

    # iterate through to test the neural network
    while curr_point < len(f):
        
                              
        # set up the first layer and evaluate it
        x.input_vals = attrs[curr_point]
        x.eval()
                             
        # set up the second layer and evaluate it
        y.input_vals = x.layer_out
        y.eval()

        # get the current error
        curr_err = err(y.layer_out[0], target[curr_point])

        # round up and down to check err
        if y.layer_out[0] >= 0.5:
            temp = 1
        else:
            temp = 0

        # increment the number incorrect if its wrong
        if(temp != target[curr_point]):
            num_incorrect += 1

        # print information about the current iteration
        print ("Current iteration: " + str(total_runs))
        print ("Current Error: " + str(curr_err) + "\n")
        data.w(curr_err)

        # iterate
        total_runs += 1
        curr_point += 1

    data.close()
    print ("Testing done! Check out the generated output files ('testing_err.txt' and 'training_err.txt')")
    print ("Error percentage on testing set: " + str(float(num_incorrect)/len(f)))

Parsing the training dataset...
Begining training the neural network:
Current iteration: 0
Current error: 0.0007022537232062254

Current iteration: 1
Current error: 0.46353727715778703

Current iteration: 2
Current error: 0.4547129872472324

Current iteration: 3
Current error: 0.0016616393517252814

Current iteration: 4
Current error: 0.4447247043004733

Current iteration: 5
Current error: 0.4320268494200592

Current iteration: 6
Current error: 0.41696921285686783

Current iteration: 7
Current error: 0.3993788484407373

Current iteration: 8
Current error: 0.008340701217459648

Current iteration: 9
Current error: 0.38231438055201156

Current iteration: 10
Current error: 0.35995946791320055

Current iteration: 11
Current error: 0.3353162749928347

Current iteration: 12
Current error: 0.30881735161791385

Current iteration: 13
Current error: 0.03130004438839254

Current iteration: 14
Current error: 0.290227686925235

Current iteration: 15
Current error: 0.2621229416176455

Current iterati

Current iteration: 350
Current error: 0.003162928286197439

Current iteration: 351
Current error: 0.0030270141848471473

Current iteration: 352
Current error: 0.002899525701730563

Current iteration: 353
Current error: 0.0027797564185262132

Current iteration: 354
Current error: 0.0026670904867843234

Current iteration: 355
Current error: 0.002561213681767859

Current iteration: 356
Current error: 0.002460944626911788

Current iteration: 357
Current error: 0.43356783533251353

Current iteration: 358
Current error: 0.003957709255371494

Current iteration: 359
Current error: 0.0037700195209277765

Current iteration: 360
Current error: 0.0035950143682090697

Current iteration: 361
Current error: 0.003431584964627074

Current iteration: 362
Current error: 0.0032787864961657046

Current iteration: 363
Current error: 0.0031356167457281163

Current iteration: 364
Current error: 0.0030015475973794808

Current iteration: 365
Current error: 0.0028758035782097667

Current iteration: 366
Current e

Current iteration: 564
Current error: 0.0033945141647568006

Current iteration: 565
Current error: 0.4226908055389561

Current iteration: 566
Current error: 0.005350360406798426

Current iteration: 567
Current error: 0.0050610562125366606

Current iteration: 568
Current error: 0.4068714266880293

Current iteration: 569
Current error: 0.007748273704179304

Current iteration: 570
Current error: 0.007258602088850019

Current iteration: 571
Current error: 0.006812292049980302

Current iteration: 572
Current error: 0.006404039868958258

Current iteration: 573
Current error: 0.006030279189996922

Current iteration: 574
Current error: 0.005687100702478149

Current iteration: 575
Current error: 0.0053715220988678255

Current iteration: 576
Current error: 0.005080989072431403

Current iteration: 577
Current error: 0.004811938095737074

Current iteration: 578
Current error: 0.004563354315369619

Current iteration: 579
Current error: 0.4112357838445529

Current iteration: 580
Current error: 0.007

Current iteration: 770
Current error: 0.003016057733312351

Current iteration: 771
Current error: 0.002889196305635251

Current iteration: 772
Current error: 0.0027700356643110082

Current iteration: 773
Current error: 0.0026580812301419056

Current iteration: 774
Current error: 0.0025524153407045684

Current iteration: 775
Current error: 0.0024527857039005657

Current iteration: 776
Current error: 0.4336703239162323

Current iteration: 777
Current error: 0.003945370606390668

Current iteration: 778
Current error: 0.0037585701360900556

Current iteration: 779
Current error: 0.003584261334614877

Current iteration: 780
Current error: 0.003421508996858197

Current iteration: 781
Current error: 0.0032696420562986894

Current iteration: 782
Current error: 0.0031268256164872707

Current iteration: 783
Current error: 0.0029932449798691445

Current iteration: 784
Current error: 0.0028678519728336117

Current iteration: 785
Current error: 0.4285795478897486

Current iteration: 786
Current erro


Current iteration: 988
Current error: 0.005090672122499985

Current iteration: 989
Current error: 0.004821266816499279

Current iteration: 990
Current error: 0.004572026196758167

Current iteration: 991
Current error: 0.004341073915119162

Current iteration: 992
Current error: 0.00412667301591992

Current iteration: 993
Current error: 0.003927729593803569

Current iteration: 994
Current error: 0.003741619982548699

Current iteration: 995
Current error: 0.003568474624242489

Current iteration: 996
Current error: 0.003406778441430553

Current iteration: 997
Current error: 0.0032558064078388026

Current iteration: 998
Current error: 0.0031139464574770986

Current iteration: 999
Current error: 0.4257574034512672

Current iteration: 1000
Current error: 0.004935883561160938

Current iteration: 1001
Current error: 0.004678122526183692

Current iteration: 1002
Current error: 0.004439458728999927

Current iteration: 1003
Current error: 0.41235702923379586

Current iteration: 1004
Current error

...

Current iteration: 1280
Current error: 0.0015758101989968732

Current iteration: 1281
Current error: 0.001535989405131808

Current iteration: 1282
Current error: 0.0014974812396061454

Current iteration: 1283
Current error: 0.0014604561839365452

Current iteration: 1284
Current error: 0.0014247586894931574

Current iteration: 1285
Current error: 0.001390468406850552

Current iteration: 1286
Current error: 0.0013571966828768173

Current iteration: 1287
Current error: 0.0013252121057879328

Current iteration: 1288
Current error: 0.001294353427873912

Current iteration: 1289
Current error: 0.0012644892696709845

Current iteration: 1290
Current error: 0.0012356652322300307

Current iteration: 1291
Current error: 0.0012078192493530419

Current iteration: 1292
Current error: 0.0011808370398121082

Current iteration: 1293
Current error: 0.0011548001275080413

Current iteration: 1294
Current error: 0.001129610557051291

Current iteration: 1295
Current error: 0.0011052024162842872

Current iter

Current iteration: 1490
Current error: 0.0004957892690914579

Current iteration: 1491
Current error: 0.0004885900625820529

Current iteration: 1492
Current error: 0.46944633834628047

Current iteration: 1493
Current error: 0.0007534420069121135

Current iteration: 1494
Current error: 0.0007400454784791629

Current iteration: 1495
Current error: 0.0007270162597589537

Current iteration: 1496
Current error: 0.0007143327247587078

Current iteration: 1497
Current error: 0.0007019539069376598

Current iteration: 1498
Current error: 0.0006899291348841797

Current iteration: 1499
Current error: 0.0006781581633875063

Current iteration: 1500
Current error: 0.0006667157524660016

Current iteration: 1501
Current error: 0.4644457613298091

Current iteration: 1502
Current error: 0.0010215389577401307

Current iteration: 1503
Current error: 0.001000514764934827

Current iteration: 1504
Current error: 0.0009801294024837518

Current iteration: 1505
Current error: 0.0009604020577963714

Current iterat


Begin testing the neural network:
Current iteration: 0
Current Error: 0.0007842766128172278

Current iteration: 1
Current Error: 0.0007842651413936828

Current iteration: 2
Current Error: 0.0007842671907557291

Current iteration: 3
Current Error: 0.0007842711370861083

Current iteration: 4
Current Error: 0.000784264355775757

Current iteration: 5
Current Error: 0.0007842759501557653

Current iteration: 6
Current Error: 0.0007842775716369716

Current iteration: 7
Current Error: 0.000784265058604918

Current iteration: 8
Current Error: 0.0007842702407444306

Current iteration: 9
Current Error: 0.0007842822021570283

Current iteration: 10
Current Error: 0.46117644641656025

Current iteration: 11
Current Error: 0.000784280338493489

Current iteration: 12
Current Error: 0.0007842749794905527

Current iteration: 13
Current Error: 0.0007842677005715995

Current iteration: 14
Current Error: 0.0007842663264558829

Current iteration: 15
Current Error: 0.0007842825932463649

Current iteration: 1

Current iteration: 547
Current Error: 0.46117543930587324

Current iteration: 548
Current Error: 0.0007842631922489898

Current iteration: 549
Current Error: 0.0007842760134021458

Current iteration: 550
Current Error: 0.000784267468543331

Current iteration: 551
Current Error: 0.4611776701762342

Current iteration: 552
Current Error: 0.46117558453983704

Current iteration: 553
Current Error: 0.0007842718327102012

Current iteration: 554
Current Error: 0.0007842676409560021

Current iteration: 555
Current Error: 0.4611782215776457

Current iteration: 556
Current Error: 0.0007842851355649605

Current iteration: 557
Current Error: 0.0007842686194060108

Current iteration: 558
Current Error: 0.0007843012132757289

Current iteration: 559
Current Error: 0.0007843193208105878

Current iteration: 560
Current Error: 0.0007842761548210035

Current iteration: 561
Current Error: 0.0007842923950014844

Current iteration: 562
Current Error: 0.0007842805142536206

Current iteration: 563
Current Erro

Current iteration: 1202
Current Error: 0.46117830673559923

Current iteration: 1203
Current Error: 0.46117938428188415

Current iteration: 1204
Current Error: 0.0007842774240701022

Current iteration: 1205
Current Error: 0.4611751405874933

Current iteration: 1206
Current Error: 0.46117374986645004

Current iteration: 1207
Current Error: 0.0007845334382295662

Current iteration: 1208
Current Error: 0.0007842642382341063

Current iteration: 1209
Current Error: 0.0007842651609443553

Current iteration: 1210
Current Error: 0.0007842639794535089

Current iteration: 1211
Current Error: 0.46117848990010035

Current iteration: 1212
Current Error: 0.0007842784861247336

Current iteration: 1213
Current Error: 0.46117700257900984

Current iteration: 1214
Current Error: 0.0007842956751165202

Current iteration: 1215
Current Error: 0.0007842713970474312

Current iteration: 1216
Current Error: 0.0007842814523809067

Current iteration: 1217
Current Error: 0.461175966889193

Current iteration: 1218
C